# Maximum flow
![title](img/ej4.png)

### EX4: Basic example MF
1. Understand the MCF model formulation.
1. Translate the math to python code. (Pay attention to model boundaries, LB / UB)
1. Understand the outputs
1. What’s the maximum flow?
1. What’s the minimum cut? 

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.latex.repr = True

# Model definitions

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale desde s y llega hasta t
beq = np.array([0, 0, 0, 0, 0, 0])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 e infinito

u = np.array([7, 1, 2, 3, 2, 1, 2, None])
l = np.zeros(8)
bounds = tuple([(l[arcs],u[arcs]) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]
 [5 0]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [1 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t-ts

Output: 
 [[ 1  1  0  0  0  0  0 -1]
 [-1  0  1  1  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  1  0  0]
 [ 0  0  0  0 -1  0  1  0]
 [ 0  0  0 -1  0 -1 -1  1]]



In [3]:
bounds

((0.0, 7),
 (0.0, 1),
 (0.0, 2),
 (0.0, 3),
 (0.0, 2),
 (0.0, 1),
 (0.0, 2),
 (0.0, None))

In [4]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])

# Inputs to optimize

In [5]:
pd.DataFrame(data=NN, columns=node_names, index=node_names)

,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,1,0,0,0,0,0


In [6]:
pd.DataFrame(data=Aeq, columns=nan_names, index=node_names)

,s->2,s->3,2->4,2->t,3->5,4->t,5->t,t->s
s,1,1,0,0,0,0,0,-1
2,-1,0,1,1,0,0,0,0
3,0,-1,0,0,1,0,0,0
4,0,0,-1,0,0,1,0,0
5,0,0,0,0,-1,0,1,0
t,0,0,0,-1,0,-1,-1,1


In [7]:
resume_df = pd.DataFrame(bounds, index=nan_names, columns=['Min bound', 'Max bound'])
resume_df['Costs'] = C

In [8]:
resume_df

,Min bound,Max bound,Costs
s->2,0.0,7.0,0
s->3,0.0,1.0,0
2->4,0.0,2.0,0
2->t,0.0,3.0,0
3->5,0.0,2.0,0
4->t,0.0,1.0,0
5->t,0.0,2.0,0
t->s,0.0,NaN,-1


In [9]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [ 0  0  0  0  0  0  0 -1] 

Columns: ['s->2' 's->3' '2->4' '2->t' '3->5' '4->t' '5->t' 't->s'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0 -1]
 [-1  0  1  1  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  1  0  0]
 [ 0  0  0  0 -1  0  1  0]
 [ 0  0  0 -1  0 -1 -1  1]] 

b_eq demand-supply vector: [0 0 0 0 0 0] 

Bounds of each X arc variable: ((0.0, 7), (0.0, 1), (0.0, 2), (0.0, 3), (0.0, 2), (0.0, 1), (0.0, 2), (0.0, None)) 



# Solve trough simplex method

In [10]:
# Optimización
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

selarcs = get_selected_arcs(res.x, nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [11]:
res.x

array([4., 1., 1., 3., 1., 1., 1., 5.])

In [12]:
print('## Results ##')
print('The raw solution will be: %s' % res.x)
# print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)

## Results ##
The raw solution will be: [4. 1. 1. 3. 1. 1. 1. 5.]
The minimum cost will be: -5.00 


In [13]:
res

     con: array([0., 0., 0., 0., 0., 0.])
     fun: -5.0
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([4., 1., 1., 3., 1., 1., 1., 5.])

# Results

In [14]:
resume_df['Flow'] = res.x

In [15]:
resume_df

,Min bound,Max bound,Costs,Flow
s->2,0.0,7.0,0,4.0
s->3,0.0,1.0,0,1.0
2->4,0.0,2.0,0,1.0
2->t,0.0,3.0,0,3.0
3->5,0.0,2.0,0,1.0
4->t,0.0,1.0,0,1.0
5->t,0.0,2.0,0,1.0
t->s,0.0,NaN,-1,5.0


## At table we can se flow trough each path.
## Path t -> s with cost -1 is neccesary to model MFC